## 🧱 Özel Ortam

---

Bu görevde, derste örnek olarak kullanılan **ızgara tabanlı navigasyon ortamının** birebir aynısını tasarlayıp uygulayacaksınız.

- Ajan ızgaranın bir köşesinde başlar.
- Amaç karşı köşeye ulaşmaktır.
- Izgarada ajanın kaçınması gereken engeller veya "delikler" bulunabilir.

---

### 🎯 Amaçlar

- 📐 **Ortamı Tanımlayın**: ajanın başlangıçtan hedefe doğru hareket ettiği bir ızgara dünyası oluşturun.
- ⚙️ **Ortam Dinamiklerini Uygulayın**: hareket ve ödül ataması kurallarını programlayın (örn. deliklere düşmek için ceza, hedefe ulaşmak için ödül).
- 👁️ **Gözlem ve Eylemleri Ayarlayın**:
    - **Gözlem alanı** → Ajanın algıladığı şeyler (örn. ızgaradaki pozisyonu)  
    - **Eylem alanı** → Ajanın yapabileceği şeyler (örn. yukarı, aşağı, sola, sağa hareket)
- 🖼️ **Render Metodu Ekleyin**: ortamı görselleştirmek için `.render()` fonksiyonu ekleyin — hata ayıklama ve ajanın davranışını anlama için faydalıdır.
- 🧩 **Özel Özellikler Ekleyin**: derste ele alınan engeller, çukurlar veya diğer özellikleri dahil ederek ortamınızı daha dinamik ve gerçekçi hale getirin.

---
Bu görev için ihtiyacımız olan tüm paketleri içe aktararak başlayalım:

In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, Dict, Tuple


import gymnasium as gym
from gymnasium import spaces
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import DQN
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env

---

### 🧩 Bölüm 1: Özel Ortam Sınıfı Oluşturma

Bu bölümde, kısmen yazılmış bir sınıfı tamamlayarak özel ortamınızı uygulayacaksınız.  
Yapı, Gymnasium'un ortam oluşturma için standart formatını takip eder.

### 🛠️ Yapacaklarınız

- Bazı temel metotları önceden yazılmış bir şablon sağlanmıştır.  
- `# CODE HERE` yazdığını gördüğünüz her yerde, eksik mantığı doldurmanız gerekir.  
- Her adımda size rehberlik etmesi için satır içi ipuçları verilmiştir.

### 🎯 Hedefiniz

- Sınıfı gerçek bir Gymnasium ortamı gibi davranacak şekilde tamamlayın.  
- Şunları yapmalı:
  - Geçerli bir gözlem ve eylem alanı tanımlamalı  
  - Ajan hareketini ve geçişleri ele almalı  
  - Ödülleri döndürmeli ve `done` bayraklarını uygun şekilde güncellemeli  
  - Çalışan bir `.reset()` ve `.step()` metodu içermeli  
  - İsteğe bağlı olarak görselleştirme için basit bir `.render()` içermeli

🧠 Her metodu anlamak için zaman ayırın — özellikle durum geçişlerinin ve ödüllerin nasıl yönetildiğini. RL ortamları burada canlanır.

📚 Ortam yapısı ve en iyi uygulamalar hakkında ayrıntılı adımlar için resmi [Gymnasium özel ortam kılavuzuna](https://gymnasium.farama.org/introduction/create_custom_env/) başvurun.

In [4]:
class CustomGridEnv(gym.Env):
    metadata = {'render_modes': ['human']}

    def __init__(self):
        # self.size = # Size of the square grid
        # self.action_space = # Define the action space to have four possible actions: right, up, left, down
        # self.agent_position = # Initial position of the agent (top-left corner)
        # self.goal_position =  # Position of the target (bottom-right corner)
        # self.hole_position =  # Position of the hole (bottom middle)
        # self.action_to_direction = # Mapping from actions to grid movements
        self.size = 3
        self.action_space = spaces.Discrete(4)
        self.agent_position = np.array([0, 0], dtype=np.int32)
        self.goal_position = np.array([2, 2], dtype=np.int32)
        self.hole_position = np.array([2, 1], dtype=np.int32)
        self.action_to_direction = {
            0: np.array([0, 1]),  # Move right
            1: np.array([-1, 0]),  # Move up
            2: np.array([0, -1]),  # Move left
            3: np.array([1, 0])   # Move down
        }

        # Define the observation space using dictionary spaces for agent and target locations
        self.observation_space = spaces.Dict({
            "agent": spaces.Box(low=0, high=self.size - 1, shape=(2,), dtype=np.int32),
            "target": spaces.Box(low=0, high=self.size - 1, shape=(2,), dtype=np.int32),
        })

    def reset(self, seed: Optional[int] = None, options: Optional[Dict] = None) -> Tuple[Dict, Dict]:
        # Reset the environment for a new episode
        super().reset(seed=seed)

        # self.agent_position = # Reset the agent's position to the starting position
        self.agent_position = np.array([0, 0], dtype=np.int32)

        # Goal position remains constant
        self.goal_position = np.array([2, 2], dtype=np.int32)

        # Generate observation and info for the current state
        observation = self._get_obs()
        info = self._get_info()
        return observation, info

    def _get_obs(self):
        # Return current observation of agent and target positions
        return {
            "agent": self.agent_position.copy(),
            "target": self.goal_position.copy()
        }

    def _get_info(self):
        # Compute the Manhattan distance between the agent and the target
        distance = np.sum(np.abs(self.agent_position - self.goal_position))
        return {"distance": distance}

    def step(self, action):
        # Apply the action to change the agent_position
        # If agent fell in hole, give -10 reward, and set done=True
        # elif agent reached goal, give +10 reward, and set done=True
        # else, give -1 reward, and set done=False
        direction = self.action_to_direction[action]

        self.agent_position = np.clip(
            self.agent_position + direction, 0, self.size - 1
        )
        # Determine if the agent has reached the hole or the target
        if np.array_equal(self.agent_position, self.hole_position):
            reward, done = -10, True  # Fall into hole
        elif np.array_equal(self.agent_position, self.goal_position):
            reward, done = 10, True   # Reach the target
        else:
            reward, done = -1, False  # Normal move

        # Collect observation and info about the current state
        observation = self._get_obs()
        info = self._get_info()
        return observation, reward, done, False, info  # 'truncated' is always False

    def render(self, mode='human'):

        # Visual representation of the grid
        grid = np.full((self.size, self.size), fill_value=' ')

        # Get x and y indexes for agent_position, goal_position, and hole_position
        agent_x, agent_y = self.agent_position
        goal_x, goal_y = self.goal_position
        hole_x, hole_y = self.hole_position

        grid[agent_x][agent_y] = 'A'
        grid[goal_x][goal_y] = 'G'
        grid[hole_x][hole_y] = 'H'

        # Print the grid with borders
        print("+---" * self.size + "+")
        for row in grid:
            print("|" + "|".join(f" {cell} " for cell in row) + "|")
            print("+---" * self.size + "+")

👇 Önceki bölümü doğru tamamladığınızı test etmek için aşağıdaki hücreyi çalıştırın. Doğru yaptıysanız, ajanın bir bölümün sonu olan hedefe veya deliğe ulaşana kadar ortamınızda rastgele hareket ettiğini göreceksiniz.

In [5]:
# Create an instance of the custom environment
env = CustomGridEnv()

# Reset the environment to its initial state and get the initial observation and info
obs, info = env.reset()
print("Initial Observation:", obs)  # Display the initial position of the agent and the target
print("Initial Info:", info)  # Display additional information such as the distance from the target

# Loop through a maximum of 100 steps
for _ in range(100):
    action = env.action_space.sample()  # Randomly sample an action from the action space
    obs, reward, done, info, _ = env.step(action)  # Apply the action and get the results
    env.render()  # Render the current state of the environment to visualize the agent's position

    # Print the current state, reward received, whether the episode is done, and any additional info
    print(f"State: {obs}, Reward: {reward}, Done: {done}, Info: {info}")

    # If the episode is finished (agent reached the goal or fell into a hole), exit the loop
    if done:
        break


Initial Observation: {'agent': array([0, 0], dtype=int32), 'target': array([2, 2], dtype=int32)}
Initial Info: {'distance': 4}
+---+---+---+
|   |   |   |
+---+---+---+
| A |   |   |
+---+---+---+
|   | H | G |
+---+---+---+
State: {'agent': array([1, 0]), 'target': array([2, 2], dtype=int32)}, Reward: -1, Done: False, Info: False
+---+---+---+
|   |   |   |
+---+---+---+
| A |   |   |
+---+---+---+
|   | H | G |
+---+---+---+
State: {'agent': array([1, 0]), 'target': array([2, 2], dtype=int32)}, Reward: -1, Done: False, Info: False
+---+---+---+
| A |   |   |
+---+---+---+
|   |   |   |
+---+---+---+
|   | H | G |
+---+---+---+
State: {'agent': array([0, 0]), 'target': array([2, 2], dtype=int32)}, Reward: -1, Done: False, Info: False
+---+---+---+
|   | A |   |
+---+---+---+
|   |   |   |
+---+---+---+
|   | H | G |
+---+---+---+
State: {'agent': array([0, 1]), 'target': array([2, 2], dtype=int32)}, Reward: -1, Done: False, Info: False
+---+---+---+
|   |   |   |
+---+---+---+
|   | A

---
## Bölüm 2: DQN Eğitimi 🤖

Bu bölümde, özel ortamınızı başlatacak ve Stable Baselines3 kütüphanesini kullanarak bir DQN ajanı ile etkileşim için hazırlayacaksınız. Temel görev, ortamınızın kütüphanenin gereksinimleriyle uyumlu olduğundan emin olmaktır, bu da onu uygun şekilde sarmalamayı içerir.

#### 📝 İzlenecek adımlar

1. 🧱 **Özel Ortamı Başlatın**: özel ortam sınıfınızın bir örneğini oluşturun.
2. 🔁 **SB3 Uyumluluğunu Sağlayın**: Stable Baselines3'ten `make_vec_env` fonksiyonunu kullanarak ortamınızı sarın, böylece kütüphanenin vektörleştirilmiş ortam gereksinimleriyle uyumlu hale getirin.
3. ⚙️ **DQN Ajanını Yapılandırın ve Eğitin**: DQN ajanını uygun hiperparametrelerle kurun ve ortamınızda eğitin.
4. 📊 **Eğitim İlerlemesini İzleyin**: Ajanın öğrenme ilerlemesini ve performansını zaman içinde gözlemlemek için günlükleme ve izleme uygulayın.
5. 💾 **Modeli kaydedin**: Eğitim sonrasında modelinizi kaydedin.

In [6]:
env = make_vec_env(lambda: CustomGridEnv(), n_envs=1)

model = DQN(
    "MultiInputPolicy",
    env,
    verbose=1,
    tensorboard_log="./dqn_custom_grid_tensorboard/"
)

model.learn(total_timesteps=25000)


model.save("dqn_custom_grid")

Using cpu device
Logging to ./dqn_custom_grid_tensorboard/DQN_1
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 23.8     |
|    ep_rew_mean      | -17.8    |
|    exploration_rate | 0.964    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 12253    |
|    time_elapsed     | 0        |
|    total_timesteps  | 95       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 19.1     |
|    ep_rew_mean      | -15.6    |
|    exploration_rate | 0.942    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 840      |
|    time_elapsed     | 0        |
|    total_timesteps  | 153      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.582    |
|    n_updates        | 13       |
----------------------------------
--------------------------

---
## 🎮 Bölüm 3: Eğitilmiş modelinizi kullanın

Eğitilmiş DQN modelinizi yükleyecek ve ortamda karar vermek için kullanacaksınız, başlangıç noktasından hedefe delikleri kaçınarak nasıl gittiğini gözlemleyeceksiniz. Bu sadece eğitimin etkinliğini doğrulamanıza izin vermeyecek, aynı zamanda ajanın karar verme sürecini görsel olarak yorumlamanıza da olanak sağlayacaktır.

### İzlenecek adımlar 📝
1. 💾 **Eğitilmiş Modeli Yükleyin**: Eğitim aşamasında kaydedilen DQN modelini alın.
2. 🔄 **Ortamı Sıfırlayın**: Navigasyon görevini sıfırdan başlatmak için ortamı başlatın.
3. 🧠 **Navigasyon Simülasyonunu Çalıştırın**: Modeli kullanarak ortamın durumlarına dayalı eylemleri tahmin edin ve ajanın attığı her adımı görselleştirin.
4. 👀 **Her Adımı Görselleştirin**: Ajanın pozisyonunu, hedefi ve herhangi bir engeli veya deliği gösteren ızgaranın basit bir görselleştirmesini uygulayın.
5. 📝 **Ajan Davranışını Analiz Edin**: Ajanın hedefe ulaşma yeteneğini gözlemleyin ve not edin ve delikleri ne kadar etkili bir şekilde kaçındığını görün.

Kodun bir kısmı zaten orada ve `# CODE HERE` yorumunu gördüğünüz her yerde kod doldurmanız gerekiyor. Başlamanız için bazı ipuçları bulacaksınız.

In [7]:
# Load the model
model = DQN.load("dqn_custom_grid")


obs = env.reset()  # Reset the environment to the initial state and get the first observation

for _ in range(200):  # Loop through a maximum of 200 steps (or until the episode ends)
    # Use the model to predict the next action to be taken, based on the current observation
    # Execute the chosen action in the environment, receive the next state and reward, and check if the episode is done
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, done, info = env.step(action)

    # Initialize a 3x3 grid filled with spaces to represent the environment visually
    grid = np.full((3,3 ), fill_value=' ')

    # Extract coordinates for the agent, goal, and hole using the current observation
    agent_x, agent_y = obs['agent'][0][0],obs['agent'][0][1]
    goal_x, goal_y = obs['target'][0][0],obs['target'][0][1]
    hole_x, hole_y = 2,1  # The hole's position is fixed in the environment

    # Update the grid with the agent's, goal's, and hole's positions
    grid[agent_x][agent_y] = 'A'  # Mark the agent's position with 'A'
    grid[goal_x][goal_y] = 'G'  # Mark the goal position with 'G'
    grid[hole_x][hole_y] = 'H'  # Mark the hole position with 'H'

    # Print the current state, reward received, if the episode is done, and any additional info
    print(f"State: {obs}, Reward: {rewards}, Done: {done}, Info: {info}")

    # Print the grid visually in the console
    print("+---" * 3 + "+")  # Top border of the grid
    for row in grid:
        print("|" + "|".join(f" {cell} " for cell in row) + "|")  # Print each row with cells separated by '|'
        print("+---" * 3 + "+")  # Separator border after each row

    # If the episode is finished (either the goal is reached or the agent fell into the hole), reset the environment
    if done:
        obs = env.reset()

State: OrderedDict({'agent': array([[0, 1]], dtype=int32), 'target': array([[2, 2]], dtype=int32)}), Reward: [-1.], Done: [False], Info: [{'distance': 3, 'TimeLimit.truncated': False}]
+---+---+---+
|   | A |   |
+---+---+---+
|   |   |   |
+---+---+---+
|   | H | G |
+---+---+---+
State: OrderedDict({'agent': array([[1, 1]], dtype=int32), 'target': array([[2, 2]], dtype=int32)}), Reward: [-1.], Done: [False], Info: [{'distance': 2, 'TimeLimit.truncated': False}]
+---+---+---+
|   |   |   |
+---+---+---+
|   | A |   |
+---+---+---+
|   | H | G |
+---+---+---+
State: OrderedDict({'agent': array([[0, 0]], dtype=int32), 'target': array([[2, 2]], dtype=int32)}), Reward: [-10.], Done: [ True], Info: [{'distance': 1, 'episode': {'r': -12.0, 'l': 3, 't': 31.599774}, 'TimeLimit.truncated': False, 'terminal_observation': {'agent': array([2, 1]), 'target': array([2, 2], dtype=int32)}}]
+---+---+---+
| A |   |   |
+---+---+---+
|   |   |   |
+---+---+---+
|   | H | G |
+---+---+---+
State: Ordere

🧠 Artık ajanınızın ortamda hareket ettiğini görmelisiniz.

Eğer ajan **belirli eylemleri tekrarlayarak takılıp kalırsa** veya hedefe ulaşamazsa, muhtemelen **yeterince uzun eğitilmediği** içindir.

Eğitim adım sayısını artırmayı deneyin ve modeli yeniden eğitin — daha uzun eğitim genellikle ajanın daha iyi bir politika öğrenmesine yardımcı olur.